In [77]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

In [78]:
train_dir = "dataset/train"
val_dir = "dataset/test"
img_size = (224, 224)
batch_size = 32

In [79]:
train_ds = image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size
)

Found 3088 files belonging to 4 classes.
Found 4 files belonging to 4 classes.


In [ ]:
# Improving Generalization Ability

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

In [ ]:
# --------------------
# 2. Load the pre-trained MobileNetV2 model (excluding the final classification layer)
# --------------------
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False, 
    weights="imagenet"
)
base_model.trainable = False   # Freeze convolutional layer parameters

In [ ]:
# --------------------
# 3. Build a new model
# --------------------
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)  # Prevent overfitting
outputs = layers.Dense(len(train_ds.class_names), activation="softmax")(x)

model = models.Model(inputs, outputs)

In [ ]:
# --------------------
# 4. Compile model
# --------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
# --------------------
# 5. Train
# --------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40
)

Epoch 1/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 27s 232ms/step - accuracy: 0.5396 - loss: 1.1521 - val_accuracy: 1.0000 - val_loss: 0.2195
Epoch 2/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 19s 194ms/step - accuracy: 0.9999 - loss: 0.1214 - val_accuracy: 1.0000 - val_loss: 0.0818
Epoch 3/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 18s 188ms/step - accuracy: 1.0000 - loss: 0.0461 - val_accuracy: 1.0000 - val_loss: 0.0454
Epoch 4/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 19s 197ms/step - accuracy: 1.0000 - loss: 0.0252 - val_accuracy: 1.0000 - val_loss: 0.0299
Epoch 5/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 18s 186ms/step - accuracy: 1.0000 - loss: 0.0165 - val_accuracy: 1.0000 - val_loss: 0.0219
Epoch 6/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 18s 186ms/step - accuracy: 1.0000 - loss: 0.0115 - val_accuracy: 1.0000 - val_loss: 0.0170
Epoch 7/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 17s 178ms/step - accuracy: 1.0000 - loss: 0.0088 - val_accuracy: 1.0000 - val_loss: 0.0134
Epoch 8/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 18s 183ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accu

In [147]:
# --------------------
# 6. Save Model
# --------------------

model.save("mobilenetv2_custom.keras")

print("✅ The model has been trained and saved: mobilenetv2_custom.keras")

✅ The model has been trained and saved: mobilenetv2_custom.keras


Using the Model

In [87]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
print(train_ds.class_names)

['stop sign', 'traffic cone', 'traffic lights', 'walker']


In [ ]:
# Read Picture
img_path = "dataset/test/1.png"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)
class_names = train_ds.class_names
print("Result:", class_names[np.argmax(pred)])

print(pred)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Result: stop sign
[[9.9909723e-01 1.8722552e-04 2.7068341e-04 4.4489323e-04]]
